# Experiments driver for NATOPS

This notebook collects the experiments performed for TIME 2025.

In [25]:
import Pkg

# activate the general environment of ModalAssociationRules.jl package
Pkg.activate(joinpath("..", "..", ".."))

# a collection of utilities
include(joinpath("..", "experiments-driver.jl"));

  Activating project at `~/.julia/dev/ModalAssociationRules`


In [26]:
# algorithm to be used for mining
miningalgo = apriori

# distance function to compare a time series and a motif of the same length; 
# we suggest to use Z-Normalized Euclidean distance if you are not interested in the scale
# but just in identifying common shapes between signals.
expdistance = (x, y) -> zeuclidean(x, y) |> first;

In [27]:
# load NATOPS dataset
X, y = load_NATOPS();

# also, insert a new column representing the difference in height between the thumb and 
# the middle finger tip of the right hand;
# this is useful to describe "All clear" and "Not clear" classes.
insertcols!(X, 25, "ΔY[Thumb r and Hand tip r]" => X[:,5]-X[:,23]);

In [28]:
# separate all the classes in separate collections

# I have command
IHCC = reduce(vcat, [X[1:30, :], X[(180+1):(180+30), :]]);
# All clear
ACC = reduce(vcat, [X[31:60, :], X[(180+31):(180+60), :]]);
# Not clear
NCC = reduce(vcat, [X[61:90, :], X[(180+61):(180+90), :]]);
# Spread wings
SWC = reduce(vcat, [X[91:120, :], X[(180+91):(180+120), :]]);
# Fold wings
FWC = reduce(vcat, [X[121:150, :], X[(180+121):(180+150), :]]);
# Lock wings
LWC = reduce(vcat, [X[151:180, :], X[(180+151):(360), :]]);

In [29]:
# name for each variable;
# this will be useful later when printing during the snippets labeling

variablenames = [
    "X[Hand tip l]", "Y[Hand tip l]", "Z[Hand tip l]",
    "X[Hand tip r]", "Y[Hand tip r]", "Z[Hand tip r]",
    "X[Elbow l]", "Y[Elbow l]", "Z[Elbow l]",
    "X[Elbow r]", "Y[Elbow r]", "Z[Elbow r]",
    "X[Wrist l]", "Y[Wrist l]", "Z[Wrist l]",
    "X[Wrist r]", "Y[Wrist r]", "Z[Wrist r]",
    "X[Thumb l]", "Y[Thumb l]", "Z[Thumb l]",
    "X[Thumb r]", "Y[Thumb r]", "Z[Thumb r]",
    "ΔY[Thumb r and Hand tip r]"
];

## I have command

In [36]:
using Serialization

In [ ]:
# run this cell if you need to create your literals, that is,
# to extract snippets from time series and give them a name.

ids = []
motifs = []
featurenames = []

# we only want to consider right hand and right elbow variables
for varid in vcat(collect(3:6), collect(10:12))

    data = reduce(vcat, IHCC[:,varid])
    S = snippets(data, 4, 10; m=10)
    Slong = snippets(data, 3, 20; m=20)

    motifs = [
        [_snippet(S,1)], [_snippet(S,2)], [_snippet(S,3)], [_snippet(S,4)],
        [_snippet(Slong,1)], [_snippet(Slong,2)], [_snippet(Slong,3)]
    ]

    for (i, motif) in enumerate(motifs)
        println("Plotting $(i)-th motif of class $(variablenames[varid])")
        _plot = plot()
        plot!(motif)
        display(_plot)

        _featurename = readline()

        push!(ids, varid)
        push!(motifs, motif)
        push!(featurenames, _featurename)
    end
end

serialize(joinpath("serialized", "NATOPS-IHCC-ids", ids))
serialize(joinpath("serialized", "NATOPS-IHCC-motifs", motifs))
serialize(joinpath("serialized", "NATOPS-IHCC-featurenames", featurenames))

In [ ]:
# run this cell if you already created your literals

ids = deserialize(joinpath("serialized", "NATOPS-IHCC-ids"))
motifs = deserialize(joinpath("serialized", "NATOPS-IHCC-motifs"))
featurenames = deserialize(joinpath("serialized", "NATOPS-IHCC-featurenames"))

SystemError: SystemError: opening file "NATOPS-IHCC-featurenames": No such file or directory

In [38]:
pwd()

"/home/mauro/.julia/dev/ModalAssociationRules/test/experiments/NATOPS"